In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc6.dev101
cdv, version 1.1.2
Python 3.10.8


- [Standard Transactions](https://chialisp.com/standard-transactions)
- [p2_delegated_puzzle_or_hidden_puzzle.py](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py)
- [p2_delegated_puzzle_or_hidden_puzzle.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.clvm)

```clojure
; A puzzle should commit to `SYNTHETIC_PUBLIC_KEY` (curried)
(SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle solution)
```

```clojure
    ; conditions = (a delegated_puzzle solution))

      (if original_public_key
          ; hidden puzzle
          (assert
              (is_hidden_puzzle_correct SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle)
              conditions
          )
          ; delegated puzzle
          (c (list AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle)) conditions)
      )
```

## Two Paths:
### 1. Hidden puzzle path: 

you can solve the hidden puzzle by revealing the `original_public_key`, the hidden puzzle in `delegated_puzzle`, and a solution to the hidden puzzle.

> The hidden puzzle functionality is not used by the official wallet. It is there to allow for future functionality as needed.

> If the hidden puzzle path is taken, the `hidden puzzle` and `original public key` will be revealed
which proves that it was hidden there in the first place.

```clojure
    ; "is_hidden_puzzle_correct" returns true iff the hidden puzzle is correctly encoded

    (defun-inline is_hidden_puzzle_correct (SYNTHETIC_PUBLIC_KEY original_public_key hidden_puzzle)
      (=
          SYNTHETIC_PUBLIC_KEY
          (point_add
              original_public_key
              (pubkey_for_exp (sha256 original_public_key (sha256tree1 hidden_puzzle)))
          )
      )
    )

    ...
    (assert
      (is_hidden_puzzle_correct SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle)
      conditions
    )
```

#### [Taproot/Trapdoor](https://www.chia.net/2021/05/27/Agrgregated-Sigs-Taproot-Graftroot/)
> Taproot is subtly different from graftroot. Its big advantage is that the trapdoor conditions can be committed to up front, without needing to coordinate with the other participants in a smart transaction to get their signatures in advance.

```
def my_mast_puzzle(solution):
    metapuzzle, metasolution = solution
    assert hash(metapuzzle) in [‘hashA’, ‘hashB’]
    return metapuzzle(metasolution)
```

<img src="https://www.chia.net/wp-content/uploads/2022/08/image-44.png" alt="Taproot Public Key Generation" width="450"/>
<img src="https://www.chia.net/wp-content/uploads/2022/08/image-43.png" alt="Taproot Clean Path Signature Generation" width="450"/>




### 2. Delegated puzzle path: 

pass in 0 or `()` for `original_public_key` if it wants to use an arbitrary `delegated_puzzle` (and `solution`) signed by the `SYNTHETIC_PUBLIC_KEY` (whose corresponding private key can be calculated if you know the private key for `original_public_key`).

```python
    # graftroot, a puzzle returns a list of conditions
    delegated_puzzle_solution = Program.to((1, condition_args))
    # we have a list of conditions, solution is ignored
    # conditions = (a delegated_puzzle solution))
    solutions = Program.to([[], delgated_puzzle_solution, []])  
```

```clojure
    ; prepend AGG_SIG_ME to the list of conditions
    (c (list AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle)) conditions)
```

#### [Graftroot](https://www.chia.net/2021/05/27/Agrgregated-Sigs-Taproot-Graftroot/)
> a delegated puzzle, as in "graftroot", which should return the desired conditions.

> Graftroot at its core is simple delegation. The simplest approach to signed transactions would be to treat the solution passed in as a list of conditions, prepend a requirement that that list must be signed by your key in the aggregated signature, and return that.
```
def my_simple_puzzle(solution):
    return [(‘aggregate signature’, ‘my_public_key’, hash(solution)] + solution
```

> Graftroot makes this just slightly more complicated by its solution containing two things, a metapuzzle and a metasolution. It calls the metapuzzle passing it the metasolution, getting a list of conditions in response. It then prepends an assertion that the metapuzzle signed by the coin’s key is in the aggregate, and returns that.
```
def my_graftroot_puzzle(solution):
    metapuzzle, metasolution = solution
    Return [(‘aggregate signature’, ‘my_public_key’, hash(metapuzzle)] +
              metapuzzle(metasolution)
```

## Hiding Hidden Puzzle

> We also want the ability to pre-commit to a puzzle without revealing it.

> If this hidden puzzle were to be curried it in, any spend (even the delegated spend case) would reveal the full puzzle including the hidden puzzle.

> ...we wouldn't be able to lock up a coin with the same puzzle anymore, or people would be able to tell that the puzzle hash is the same and spend it without our consent.

> We can attempt to solve this problem by hashing the hidden puzzle. However, this has a similar problem in that if you spend the hidden case even once, people could see any identical puzzle hashes later and spend them without your consent.

> We need the puzzle to be hidden, but also have some entropy that keeps it unique to us.

In [2]:
original_sk = PrivateKey.from_bytes(bytes.fromhex("31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9"))
original_pk = original_sk.get_g1()
print(f" original sk: {bytes(original_sk).hex()}")
print(f" original pk: {original_pk}")

 original sk: 31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9
 original pk: b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84


### Synthentic keys
> The solution that the standard transaction uses is to **derive a new private key from the hidden puzzle and the original public key that can sign for the delegated spend case**.

> This is known as the `synthetic_offset`:

`synthetic_offset = sha256(original_public_key + hidden_puzzle_hash);`

> We then calculate the public key of this new private key (i.e. `synthetic_offset`), and add it to our existing original public key.

> This is known as the `synthetic_public_key`:

`synthentic_public_key = original_public_key + synthetic_offset_pubkey`

### `synthentic_public_key = original_public_key + synthetic_offset_pubkey`
> You may wonder why we add the public key from our derived private key to the original public key when it's already part of the derivation. **This is because we use the synthetic public key to verify the signature of our delegated spends as well**.

```clojure
AGG_SIG_ME SYNTHETIC_PUBLIC_KEY (sha256tree1 delegated_puzzle))
```

> When you add two public keys, the sum of their private keys gives the private key for the resulting public key. If we didn't add the original public key then anyone who knew the hidden puzzle could derive the synthetic private key and could then perform delegated spends! Adding the original public key ensures that there is still a secret component of the synthetic private key, even though half of it can be known.

If others know that we use a puzzle like `DEAFULT_HIDDEN_PUZZLE`, they can easily get `synthetic_offset` because they also know `original_pk` and they can sign the message verifiable by `synthetic_pk`! Adding `original_pk` will prevent that because others won't know `original_pk`.

> This secret component is the **private key** for the original public key.

- `original_pk` is **public**.
- `original_sk` is **private**.
- `hidden puzzle hash` usually is not known until the puzzle is revealed (or known if it's a default `DEAFULT_HIDDEN_PUZZLE`)
- Once `hidden puzzle hash` is known, the `synthetic_pk` is known because `original_pk` is known.
- `synthetic_sk` is not known (can't sign the message) unless you know `original_sk`.

```python
# sum of pks
synthetic_pk = original_pk + synthetic_offset_pk

# sum of sks
synthetic_sk = original_sk + synthetic_offset_sk

# pk of sum of sks = sum of pks
original_pk + synthetic_offset_pk = synthetic_sk.get_g1()
```

More about [synthetic keys](./synthetic-keys.ipynb).

### Spend
> This technique is also neat because it allows us to hide the **hidden puzzle** in a piece of information that was already necessary for the delegated spend. It's impossible to guess what the hidden puzzle is (because of original_pk), even if it's a standard hidden puzzle. It's even hard to tell if there's a hidden puzzle at all. All of this can contribute to the overall privacy.

> For example, if two parties agree to lock up some coins with a hidden puzzle together, you can share pubkeys and verify that information on the blockchain without revealing anything to the network. 

> Then, if you both agree that the coins can be spent with the hidden puzzle if either party is dishonest, you can trustlessly delegated spend the coins to the correct destinations and it's impossible to tell that they are not just normal everyday spends.

When a coin is spent, `puzzle` and `original_pk`, `hidden_puzzle`, and `solution_for_hidden_puzzle` are known.

The coin with the same puzzle hash can be spent the same way.

```python
solution = Program.to([original_pk, hidden_puzzle, solution_for_hidden_puzzle])  
```

```clojure
; hidden puzzle
; conditions = (a hidden_puzzle solution_for_hidden_puzzle))
; conditions should include AGG_SIG_ME
(assert
  (is_hidden_puzzle_correct SYNTHETIC_PUBLIC_KEY original_public_key delegated_puzzle)
  conditions
)
```


### Spend Hidden Puzzle

In [3]:
message = "hidden puzzle message"
message_hash = std_hash(bytes(message, "utf-8"))
hidden_puzzle = Program(
    compile_clvm_text(
'''
(mod (PK)
    (include condition_codes.clib)
    (list
        (list CREATE_COIN_ANNOUNCEMENT (sha256 "{message}"))
        (list AGG_SIG_ME PK (sha256 "{message}"))
    )
)
'''.format(message=message), search_paths=["/Users/karlkim/kimsk/chia-concepts/shared"]
    )
).curry(original_pk)

print_program(hidden_puzzle)
print_program(hidden_puzzle.run([]))

(a (q 2 (q 4 (c 6 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2)) (c (c 4 (c 5 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2))) ())) (c (q 50 . 60) 1)) (c (q . 0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84) 1))
((60 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2) (50 0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2))


In [4]:
synthetic_pk = calculate_synthetic_public_key(original_pk, hidden_puzzle.get_tree_hash())

print(f"synthetic_pk:\t{synthetic_pk}")

standard_hidden_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_synthetic_public_key(synthetic_pk)
standard_hidden_ph = standard_hidden_puzzle.get_tree_hash()
print(f"standard_hidden_ph:\t{standard_hidden_ph}")

synthetic_pk:	adfc6db83f6396c29bb3135b35d7ea908e58bb17ff6b992892b712523beb96b11ea14144ba64142cf50b3711db1cb284
standard_hidden_ph:	09bf12f23502612e4bdf26c792d10129bb762207e341b7f8d66cb3a5583c816d


In [5]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")
coin = Coin(
    parent_coin_info,
    standard_hidden_ph,
    1
)

solution = Program.to([original_pk, hidden_puzzle, []])

coin_spend = CoinSpend(
    coin,
    standard_hidden_puzzle,
    solution
)

synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    original_sk,
    standard_hidden_ph
)

sig =  AugSchemeMPL.sign(original_sk,
    (
        message_hash
        + coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

assert AugSchemeMPL.verify(
    original_pk, 
    (
        message_hash
        + coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    ),
    sig)

spend_bundle = SpendBundle(
    [coin_spend],
    sig
)

print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0xa0f5b5845b51057c2976e0e78681cbfdf103a204e4a6361e0a9788dfa6ec2cae5207f7caed8aaa4aa0efad05fb6a41690f710cebe98ea30faf9dc62070c711279de84686270c5180d583390a9697963f828e229167bd88da33f6b13ac208c7e0",
    "coin_spends": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0x09bf12f23502612e4bdf26c792d10129bb762207e341b7f8d66cb3a5583c816d"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bfff

### Get Synthetic PK from Puzzle

In [6]:
print("standard_hidden_puzzle:")
print_program(standard_hidden_puzzle)
result = standard_hidden_puzzle.uncurry()
puzzle = result[0]
synthetic_pk = result[1].first()
print("\nstandard txn puzzle:")
print_program(puzzle)
print("\nsynthetic pk:")
print_program(synthetic_pk)

standard_hidden_puzzle:
(a (q 2 (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1)) (c (q . 0xadfc6db83f6396c29bb3135b35d7ea908e58bb17ff6b992892b712523beb96b11ea14144ba64142cf50b3711db1cb284) 1))

standard txn puzzle:
(a (q 2 (i 11 (q 2 (i (= 5 (point_add 11 (pubkey_for_exp (sha256 11 (a 6 (c 2 (c 23 ()))))))) (q 2 23 47) (q 8)) 1) (q 4 (c 4 (c 5 (c (a 6 (c 2 (c 23 ()))) ()))) (a 23 47))) 1) (c (q 50 2 (i (l 5) (q 11 (q . 2) (a 6 (c 2 (c 9 ()))) (a 6 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))

synthetic pk:
0xadfc6db83f6396c29bb3135b35d7ea908e58bb17ff6b992892b712523beb96b11ea14144ba64142cf50b3711db1cb284


### Get Synthetic PK from Solution

In [7]:
print("solution:")
print_program(solution)
original_pk = G1Element.from_bytes(hexstr_to_bytes(disassemble(solution.first())))
print("\noriginal_pk:")
print(original_pk)

hidden_puzzle = solution.rest().first()
print("\nhidden puzzle:")
print_program(hidden_puzzle)
hidden_puzzle_hash = hidden_puzzle.get_tree_hash()
print("\nhidden puzzle hash:")
print(hidden_puzzle_hash)

synthetic_pk = calculate_synthetic_public_key(original_pk, hidden_puzzle_hash)
print("\nsynthetic pk:")
print(synthetic_pk)

solution:
(0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84 (a (q 2 (q 4 (c 6 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2)) (c (c 4 (c 5 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2))) ())) (c (q 50 . 60) 1)) (c (q . 0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84) 1)) ())

original_pk:
b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84

hidden puzzle:
(a (q 2 (q 4 (c 6 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2)) (c (c 4 (c 5 (q 0x07ac75e2dcf3cd9c93690a20c2ff064c2c2be05230ca5c665f3335c70ad761a2))) ())) (c (q 50 . 60) 1)) (c (q . 0xb87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84) 1))

hidden puzzle hash:
d7750a1c9aeb04d926d680d33db8b017df60bc6851df8009623d419009468bbc

synthetic pk:
adfc6db83f6396c29bb3135b35d7ea908e

### Spend Delegated Puzzle

In [8]:
synthetic_pk = calculate_synthetic_public_key(original_pk, DEFAULT_HIDDEN_PUZZLE_HASH)
standard_delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_synthetic_public_key(synthetic_pk)
standard_delegated_ph = standard_delegated_puzzle.get_tree_hash()
print(standard_delegated_ph)

879f4dc5ec5d30c6b6aec9a8531d2ec209bd618da967a4a462dececf9184c767


In [9]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")
coin = Coin(
    parent_coin_info,
    standard_delegated_ph,
    1
)

conditions = [
    [
        ConditionOpcode.CREATE_COIN_ANNOUNCEMENT,
        message_hash
    ]
]

delegated_puzzle: Program = puzzle_for_conditions(conditions) 
solution = solution_for_conditions(conditions)

coin_spend = CoinSpend(
    coin,
    standard_delegated_puzzle,
    solution
)

    
synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    original_sk,
    DEFAULT_HIDDEN_PUZZLE_HASH
)

sig =  AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

spend_bundle = SpendBundle(
    [coin_spend],
    sig
)

print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0x865040e3d472ae25c80fb61a98395b0c41d948e942b944f05b202c8e447973d26ef6ad6f0cdc133f35a585740182ec85167d8404f02ce29c053647d26bd86cf960156f507911a778457c226783cd157744182d6dfe458c72f351a654d3f56dfe",
    "coin_spends": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0x879f4dc5ec5d30c6b6aec9a8531d2ec209bd618da967a4a462dececf9184c767"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bfff